# Projeto 1 - Ciência dos Dados

Nome: Arthur Fuhrken 

Nome: Pedro Mahfuz

Nome: Rodrigo Furukawa

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
import emoji
from math import *

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\osbor\Documents\2021.1\Cdados DP\Projeto 1


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
filename = 'ruffles.xlsx'

In [4]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Valor
0,uma criancinha deu um ruffles pra ela,1
1,rt @favskyliex: 📖 sorteio 📖\n\npara concorrer ...,0
2,"@_adryelleac @americanascom aí sim, adry! imag...",2
3,@b_ruffles eu vo para de estuda também,0
4,assimmm completamente eu arrumando as parede d...,0


In [5]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Valor
0,@vulgo_ruffles amo vc 🥰🥰,0
1,rt @igoritonobrasil: 🏐💕 amor doce 💕🏐\n#haikyuu...,0
2,rt @bonecker_: incrível que eu taco o fodase p...,2
3,"[9:55 pm, 17/03/2021] alexia: eu comi só ruffl...",1
4,hj não foi um bom dia,0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

O nosso produto é a famosa batata "Ruffles", consideramos como tweets relevantes os que falassem do produto de uma maneira pejorativa ou positiva. Consideramos como irrelevantes, os tweets que falavam do produto, mas não agregavam nada a ele. Por fim, consideramos muito irrelevantes os tweets que não faziam qualquer tipo de menção ao produto ou que continham conteúdo completamente desagregado ao valor dele.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [6]:
#Criando classes "cleanup", usadas para deixar as strings mais padronizadas.

def cleanup_pontuacao(tweets):
    p = '[!-.:?;"\n"()''"",_%$®|=“”—/]'
    padrao = re.compile(p)
    remove_ponto = re.sub(padrao,' ',tweets)
    espaco = ' '
    padrao = re.compile(espaco)
    tweets_espaco = re.sub(padrao, ' ', remove_ponto)
    return tweets_espaco
    
def cleanup_espacos(tweets):    
    espacos = ' '
    padrao = re.compile(espacos)
    remove_espaco = re.sub(padrao,' ', tweets)
    return remove_espaco

def cleanup_link(tweets):
    link = r'http[^\s]*'
    padrao = re.compile(link)
    remove_link = re.sub(padrao,'',tweets)
    return remove_link

trein_str=str()
for n in train['Treinamento']:
    trein_str+=' '+n



trein = trein_str.lower()
trein1 = cleanup_pontuacao(trein)
trein2 = cleanup_espacos(trein1)
trein_limpo = cleanup_link(trein2)

In [7]:
#Usando UNICODE emoji para separar os emojis

def seleciona_emoji(character):
        return character in emoji.UNICODE_EMOJI 
    
def character_tem_emoji(texto):
    return ''.join(' ' + char + ' ' if seleciona_emoji(char) else char for char in texto).strip()    

trein_f = character_tem_emoji(trein_limpo)

In [8]:
#Criando um pd.Series com todas as palavras e suas frequencias relativas

palavras = trein_f.split()

total = pd.Series(palavras)
total.value_counts(True)

ruffles       0.046613
de            0.036663
e             0.025896
que           0.018400
a             0.017582
                ...   
p57fe0mpqd    0.000136
sessão        0.000136
bota          0.000136
bate          0.000136
abzcvhyyo5    0.000136
Length: 2186, dtype: float64

In [9]:
train['Valor']=train['Valor'].astype('category')
train['Valor'].cat.categories = ["Muito Irrelevante","Irrelevante","Relevante"]

train['Valor'].value_counts(True)

Muito Irrelevante    0.396
Irrelevante          0.330
Relevante            0.274
Name: Valor, dtype: float64

In [10]:
filtro_0 = train['Valor'] == 'Muito Irrelevante'
muito_irrelevante = train.loc[filtro_0,:]

filtro_1 = train['Valor'] == 'Irrelevante'
irrelevante = train.loc[filtro_1,:]

filtro_2 = train['Valor'] == 'Relevante'
relevante = train.loc[filtro_2,:]

In [11]:
#Limpando relevantes da tabela Treinamento com as funções criadas no inicio

relevantes_clean=str()
for n in relevante['Treinamento']:
    relevantes_clean+=' '+n
    
relevantes = relevantes_clean.lower()
relevantes1 = cleanup_pontuacao(relevantes)
relevantes2 = cleanup_espacos(relevantes1)
relevantes_limpo = cleanup_link(relevantes2)
relevantes_emoji = character_tem_emoji(relevantes_limpo)

palavras1 = relevantes_emoji.split()
total1 = pd.Series(palavras1)

In [12]:
#Limpando irrelevantes da tabela Treinamento com as funções criadas no inicio

irrelevantes_clean=str()
for n in irrelevante['Treinamento']:
    irrelevantes_clean+=' '+n
    
irrelevantes = irrelevantes_clean.lower()
irrelevantes1 = cleanup_pontuacao(irrelevantes)
irrelevantes2 = cleanup_espacos(irrelevantes1)
irrelevantes_limpo = cleanup_link(irrelevantes2)
irrelevantes_emoji = character_tem_emoji(irrelevantes_limpo)

palavras2 = irrelevantes_emoji.split()
total2 = pd.Series(palavras2)

In [13]:
#Limpando m_irrelevantes da tabela Treinamento com as funções criadas no inicio

m_irrelevantes_clean=str()
for n in muito_irrelevante['Treinamento']:
    m_irrelevantes_clean+=' '+n
    
m_irrelevantes = m_irrelevantes_clean.lower()
m_irrelevantes1 = cleanup_pontuacao(m_irrelevantes)
m_irrelevantes2 = cleanup_espacos(m_irrelevantes1)
m_irrelevantes_limpo = cleanup_link(m_irrelevantes2)
m_irrelevantes_emoji = character_tem_emoji(m_irrelevantes_limpo)

palavras3 = m_irrelevantes_emoji.split()
total3 = pd.Series(palavras3)

In [14]:
#Criando lista com palavras limpas relevantes

relevantes = []

for i in relevante['Treinamento']:
    relevantes_clean = i.lower()
    relevantes1 = cleanup_pontuacao(relevantes_clean)
    relevantes2 = cleanup_espacos(relevantes1)
    relevantes_limpo = cleanup_link(relevantes2)
    relevantes_emoji = character_tem_emoji(relevantes_limpo)
    
    relevantes.append(relevantes_emoji)

In [15]:
#Criando lista com palavras limpas irrelevantes

irrelevantes = []

for i in irrelevante['Treinamento']:
    irrelevantes_clean = i.lower()
    irrelevantes1 = cleanup_pontuacao(irrelevantes_clean)
    irrelevantes2 = cleanup_espacos(irrelevantes1)
    irrelevantes_limpo = cleanup_link(irrelevantes2)
    irrelevantes_emoji = character_tem_emoji(irrelevantes_limpo)
    
    irrelevantes.append(irrelevantes_emoji)

In [16]:
#Criando lista com palavras limpas muito irrelevantes

m_irrelevantes = []

for i in muito_irrelevante['Treinamento']:
    m_irrelevantes_clean = i.lower()
    m_irrelevantes1 = cleanup_pontuacao(m_irrelevantes_clean)
    m_irrelevantes2 = cleanup_espacos(m_irrelevantes1)
    m_irrelevantes_limpo = cleanup_link(m_irrelevantes2)
    m_irrelevantes_emoji = character_tem_emoji(m_irrelevantes_limpo)
    
    m_irrelevantes.append(m_irrelevantes_emoji)

In [17]:
#Criando lista com todas as palavras limpas

todas_palavras = []

for i in train['Treinamento']:
    todas_clean = i.lower()
    todas1 = cleanup_pontuacao(todas_clean)
    todas2 = cleanup_espacos(todas1)
    todas_limpo = cleanup_link(todas2)
    todas_emoji = character_tem_emoji(todas_limpo)
    
    todas_palavras.append(todas_emoji)

In [18]:
# Somando palavras de cada categoria

rel = total1.value_counts()
total_rel = total1.value_counts().sum()

irrel = total2.value_counts()
total_irrel = total2.value_counts().sum()

m_irrel = total3.value_counts()
total_m_irrel = total3.value_counts().sum()

total_palavras = total.value_counts()
total_palavras_soma = total.value_counts().sum()

In [19]:
# Smoothing para cada categoria

A = len(total_palavras)
a = 1

smooth_0_rel = rel + a
smooth_1_rel = total_rel + A

smooth_0_irrel = irrel + a
smooth_1_irrel = total_irrel + A

smooth_0_m_irrel = m_irrel + a
smooth_1_m_irrel = total_m_irrel + A

In [20]:
# P(palavra|"Categoria")

P_pal_rel = smooth_0_rel/smooth_1_rel
P_pal_irrel = smooth_0_irrel/smooth_1_irrel
P_pal_m_irrel = smooth_0_m_irrel/smooth_1_m_irrel

In [21]:
p_palavra_r=1.0

p_palavra_i=1.0

p_palavra_m = 1.0

for i in list(P_pal_rel):
    p_palavra_r+=log(i)


for i in list(P_pal_irrel):
    p_palavra_i=log(i)


for i in list(P_pal_m_irrel):
    p_palavra_m+=log(i)


In [22]:
p_relevantes = []
p_irrelevantes = []
p_m_irrelevantes = []

for i in relevantes:
    p = log(train["Valor"].value_counts(True)[2])
    for palavra in i.split():
        if palavra in P_pal_rel:
            p += log(P_pal_rel[palavra])
        else:
            p += log(a/total_rel + A*a)
    p_relevantes.append(p)


for i in irrelevantes:
    p = log(train["Valor"].value_counts(True)[1])
    for palavra in i.split():
        if palavra in P_pal_irrel:
            p += log(P_pal_irrel[palavra])
        else:
            p += log(a/total_irrel + A*a)
    p_irrelevantes.append(p)
    

for i in m_irrelevantes:
    p = log(train["Valor"].value_counts(True)[0])
    for palavra in i.split():
        if palavra in P_pal_m_irrel:
            p += log(P_pal_m_irrel[palavra])
        else:
            p += log(a/total_m_irrel + A*a)
    p_m_irrelevantes.append(p)
    


In [23]:
# Criando dicionários com keys = palavra, e values = probabilidade, de cada categoria

d_relevante = {}
for i in range(len(relevantes)):
    d_relevante[relevantes[i]] = p_relevantes[i]


d_irrelevante = {}
for i in range(len(irrelevantes)):
    d_irrelevante[irrelevantes[i]] = p_irrelevantes[i]


d_m_irrelevante = {}
for i in range(len(m_irrelevantes)):
    d_m_irrelevante[m_irrelevantes[i]] = p_m_irrelevantes[i]
    

# Criando código que identifica se uma palavra de ser considerada Relevante, Irrelevante ou Muito irrelevante e adiciona em um dicionario

    
d_final = {}

for k_r,v_r in d_relevante.items():
    for k_i,v_i in d_irrelevante.items():
        for k_m,v_m in d_m_irrelevante.items():
            
            if (k_r == k_i) and (k_r != k_m):
                if (v_r > v_i):
                    d_final[k_r] = "Relevante"
                elif (v_i > v_r):
                    d_final[k_i] = "Irrelevante"
                    
            elif (k_r != k_i) and (k_r == k_m):
                if (v_r > v_m):
                    d_final[k_r] = "Relevante"
                elif (v_m > v_r):
                    d_final[k_m] = "Muito irrelevante"
                    
            elif (k_i == k_m) and (k_i != k_r):
                if (v_i > v_m):
                    d_final[k_i] = "Irrelevante"
                elif (v_m > v_i):
                    d_final[k_m] = "Muito irrelevante"
                    
            elif (k_i == k_m) and  (k_i == k_r):
                if (v_r > v_i) and (v_r > v_m):
                    d_final[k_r] = "Relevante"
                elif (v_i > v_r) and (v_i > v_m):
                    d_final[k_i] = "Irrelevante"
                elif (v_m > v_r) and (v_m > v_i):
                    d_final[k_m] = "Muito irrelevante"

            elif (k_r!=k_i) and (k_r!=k_m) and (k_m!=k_i):
                if k_r not in d_final:
                    d_final[k_r] = 'Relevante'

                elif k_i not in d_final:
                    d_final[k_i] = 'Irrelevante'

                elif k_m not in d_final:
                    d_final[k_m] = 'Muito irrelevante'


In [24]:
d_total ={}

for i in todas_palavras:
    for k_f,v_f in d_final.items():
        if k_f == i:
            d_total[i] = v_f
            
lista_final = []

for i in d_total.values():
    lista_final.append(i)
    
Naive_Bayes= pd.Series(lista_final)
train['Naive Bayes'] = Naive_Bayes 

lista_relevancia = []
for i in d_final.values():
    lista_relevancia.append(i)

##### ___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [25]:
test['Valor'] = test['Valor'].astype('category')

test['Valor'].cat.categories = ["Muito irrelevante","Irrelevante","Relevante"]

In [26]:
# Verificando performance do classificador com a base de dados "Teste"

l_teste = []

for i in test["Teste"]:
    t1 = i.lower()
    t2 = cleanup_pontuacao(t1)
    t3 = cleanup_espacos(t2)
    t4 = cleanup_link(t3)
    t = character_tem_emoji(t4)
    l_teste.append(t)
    
    
v = pd.Series(l_teste)
test['Naive Bayes'] = v


d_final_t = {}

for i in l_teste:
    for k,v in d_final.items():
        if k == i:
            d_final_t[i] = v


relevancia = []

for i in d_final_t.values():
    relevancia.append(i)
    
test["Relevancia Naive Bayes"] = pd.Series(lista_relevancia)

teste = pd.crosstab(test["Valor"],test["Relevancia Naive Bayes"],normalize= True)
teste

Relevancia Naive Bayes,Irrelevante,Muito irrelevante,Relevante
Valor,,,
Muito irrelevante,0.072,0.272,0.004
Irrelevante,0.064,0.224,0.000
Relevante,0.072,0.292,0.000


In [27]:
falso_positivo_r = 36.4
verdadeiro_positivo_r = 0

falso_positivo_i = 22.4
verdadeiro_positivo_i = 6.4

falso_positivo_m = 7.6
verdadeiro_positivo_m = 27.2

In [28]:
# Positivos falsos e verdadeiros da categoria Relevante

print('Verdadeiro Positivo Relevantes = ',verdadeiro_positivo_r)
print('Falso Positivo Relevantes = ',falso_positivo_r)

Verdadeiro Positivo Relevantes =  0
Falso Positivo Relevantes =  36.4


In [29]:
# Positivos falsos e verdadeiros da categoria Irrelevante

print('Verdadeiro Positivo Irrelevantes = ',verdadeiro_positivo_i)
print('Falso Positivo Irrelevantes = ',falso_positivo_i)

Verdadeiro Positivo Irrelevantes =  6.4
Falso Positivo Irrelevantes =  22.4


In [30]:
# Positivos falsos e verdadeiros da categoria Muito irrelevante

print('Verdadeiro Positivo Muito irrelevantes = ',verdadeiro_positivo_m)
print('Falso Positivo Muito irrelevantes = ',falso_positivo_m)

Verdadeiro Positivo Muito irrelevantes =  27.2
Falso Positivo Muito irrelevantes =  7.6


In [31]:
# Calculando a acurácia do nosso classificador

a_t = (teste.iloc[1,0]+teste.iloc[2,1])*100
print("A acurácia do classificador é ",a_t,"%.")

A acurácia do classificador é  35.6 %.


___
### Concluindo

Podemos concluir que, de acordo com a acurácia do nosso classificador, o mesmo não é muito preciso. Apesar disso, vale ressaltar o processo de construção do classificador. Primeiramente, escolhemos um produto - Ruffles -, selecionamos um número abundante de tweets e os classificamos em três categorias: "relevantes", "irrelevantes" e "muito irrelevantes". Em seguida, limpamos todos os tweets, removendo os caracteres considerados "desnecessários", pois não ajudariam na classificação. De tal maneira, links, e espaços extras  foram removidos, dando lugar à espaçamentos necessários entre emojis e palavras. Assim, fizemos uma separação dos tweets em listas de acordo com cada categoria e demos valores subsequentes relacionando cada palavra e sua frequência. 

Com a primeira classificação feita, criamos um "smoothing", que impede que uma palavra desconhecida zere a conta da classificação. Como etapa final da classificação, fizemos um código para receber tweets e de acordo com cada palavra, atribuímos um valor referente às etapas passadas. Com esses valores o classificador faz uma conta e decide a qual categoria o tweet pertence. Um dos possíveis motivos pelo qual a acurácia ficou tão baixa se deve a forma como classificamos cada frase no começo, pois não levamos em consideração palavras que tendem a descrever a relevância do tweet para o produto. Não poderiamos utilizar o nosso classificador para gerar mais amostras de treinamento pois ele não tem uma acurácia de 100% e isso acarretaria em classificações erronêas que acabariam prejudicando a acurácia do classificador ainda mais. Uma maneira de melhorar nosso calssificador seria se seguissemos um padrão melhor ao classificar nossa base de treinamento, por exemplo, se classificassemos cada tweet prestando atenção tanto em sua relevância pelo sentido da frase quanto a relevância possivel de cada palavra. O classificador poderia ser utilizado também por seguros de saúde, em relação a escolha de quem eles aceitam, pois podem ver se o cliente tem "alta ou baixa" probabilidade de necessitar de atendimento médico frequente, ou também poderia ser utilizado por algum tipo de segurança ou polícia para saber se pessoas tem "alta ou baixa" tendência a cometer um crime de acordo com suas ações na internet. Se formos pensar em comentários sarcásticos e com dupla negação, podemos concluir que estes podem atrapalhar o classificador atual se não forem identificados, portanto se conseguíssemos melhorar ele ao ponto de conseguir identificar tais comentários, teriamos milhares de funções para ele, mas para isso, necessitariamos de mais fundos e tempo.

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)


___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**